# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [302]:
text = open('peace.txt', 'r', encoding='utf-8').read()[2:]
len(text)

3227578

In [303]:
import string
import re
import time
from collections import Counter

def preprocess_text(text):
    # TODO
    # make lowercase
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    text = text.lower()
    text = re.sub(r'[^a-zA-Z_\.]', r' ', text)
    text = re.sub(r'\s+', r' ', text)
    return text

#%%time

text = preprocess_text(text)
len(text)
#assert len(text) == 3141169

3139299

In [304]:
text = text.split('.')
text = [x.strip() for x in text]

In [306]:
from collections import Counter
import nltk
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        
        text_line = ''
        for t in text:
            text_line += t
        list_text = list(text_line)
        self.itos = [a for a in set(list_text)]
        indices = [i for i in range(len(self.itos))]
        self.stoi = {k:v for k, v in zip(self.itos, indices)}
        bigram_freqs = Counter()
        for t in text:
            bigram_freqs.update(zip(t, t[1:]))
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            try:
                new_token = bigram_freqs.most_common(1)[0][0]# most common bigram in the text
            except:
                break
            new_id = len(self.itos)
            new_token = ''.join([str(i) for i in new_token])
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            # find occurences of the new_token in the text and replace them with new_id
            new_text = []
            for t in text:
                new_t = []
                i = 0
                while i < len(t)-1:
                #for i in range(len(t)-1):
                    el, next_el = t[i], t[i+1]
                    bigram = [str(el), str(next_el)]
                    bigram = ''.join(bigram)
                    if bigram == new_token:
                        new_t.append(new_token)
                        i += 2
                    else:
                        new_t.append(el)
                        i += 1
                new_text.append(new_t)
            text = new_text
            bigram_freqs = Counter()
            for t in text:
                bigram_freqs.update(zip(t, t[1:]))
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        keys = list(self.stoi.keys())
        keys_len_sorted = sorted(keys, key=len, reverse=True)
        new_texts = []
        for t in text:
            new_t = []
            while len(t) > 0:
                for k in keys_len_sorted:
                    if t.startswith(k):
                        len_k = len(k)
                        new_t.append(self.stoi[k])
                        t = t[len_k:]

            new_texts.append(new_t)
        text = new_texts    
            
                    
        #for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
            #text = # TODO
            
        return text
    
    def decode_token(self, tok): 
        """
        tok: int or tuple
        """
        result = self.itos[tok]# TODO
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))

In [307]:
vocab_size = 100
bpe = BPE(vocab_size)
var = bpe.fit(text)

In [308]:
tokenized_text = bpe.transform(text)

In [309]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [310]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size + 1

  
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = Counter()# TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        freq_list = []
        freq_sum = 0
        for w in range(self.vocab_size):
            w_counter = self.get_proba(a, b, w, tau=tau)
            freq_list.append(w_counter)
            freq_sum += w_counter
        result = [i/freq_sum for i in freq_list]
        return np.array(result)
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        trigram_proba =  self.proba[(a, b, c)]
        result = (trigram_proba + self.delta) ** (1/tau) # TODO approximate probability by counters
        return result
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        for t in text:
            t = [start_token] + [t_el for t_el in t] + [end_token]
            self.proba.update((zip(t, t[1:], t[2:])))# TODO count 3-grams in the text
        return self

lm = LM(vocab_size, 1)
lm.fit(tokenized_text)

In [269]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    beam = [(input_seq, 0)] # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len):
        candidates = []
        candidates_proba = []
        for snt, snt_proba in beam:
            if np.argmax(snt[-1]) == end_token:
                continue
            else:
                proba = lm.infer(snt[-2], snt[-1], tau) # probability vector of the next token
                best_k = proba.argsort()[-k:][::-1] # top-k most probable tokens
                # TODO update candidates' sequences and corresponding probabilities
                candidates += [snt + [int(x)] for x in best_k]
                candidates_proba += [snt_proba + x for x in np.log(proba[best_k])]

        beam = [(candidates[i], candidates_proba[i]) for i in np.argsort(candidates_proba).astype(int)[-k:][::-1]]
                # select top-k most probable sequences from candidates
    return beam
    

In [321]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(f'sequence: {bpe.decode(snt)}, probability: {np.exp(snt_proba)}')

sequence: horse of the was and , probability: 0.2625549047590843
sequence: horse of the said no, probability: 0.14853518166872237
sequence: horse the the was , probability: 0.14273488000514523
sequence: horse of the saing, probability: 0.09892398490650087
sequence: horse the the said , probability: 0.0720614794748892
sequence: horse and the was , probability: 0.05750507492785594
sequence: horse the the sain, probability: 0.04748967620059593
sequence: horse and the said , probability: 0.02903215231249907
sequence: horse and the sain, probability: 0.019132656209305265
sequence: horse of the was of , probability: 0.016910725826546534


In [323]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(f'sequence: {bpe.decode(snt)}, probability: {np.exp(snt_proba)}')

sequence: here the was and , probability: 0.41367929316290586
sequence: here the said no, probability: 0.2340307792723301
sequence: here the saing, probability: 0.1558637961478164
sequence: here she was and , probability: 0.027144450436631204
sequence: here the was of , probability: 0.026644396962290703
sequence: here the count , probability: 0.025227229051286458
sequence: here the was h, probability: 0.019381973344904452
sequence: here the had th, probability: 0.015484186676600209
sequence: here she said no, probability: 0.015356429469875086
sequence: here she saing, probability: 0.010227335908093178


In [324]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(f'sequence: {bpe.decode(snt)}, probability: {np.exp(snt_proba)}')

sequence: whation of the was, probability: 7.779316442905634e-06
sequence: whattle of the was, probability: 6.339932447711428e-06
sequence: what of the the had, probability: 5.526646780577358e-06
sequence: what of the the wa, probability: 4.777317364557427e-06
sequence: what ing ing h, probability: 4.463774536636801e-06
sequence: whation of the sai, probability: 3.7627873219074915e-06
sequence: what of the the sa, probability: 3.190163551834125e-06
sequence: what ing ing i, probability: 2.307784299344787e-06
sequence: what of the the wi, probability: 2.1292568344294423e-06
sequence: what ing ing and , probability: 1.6954624897081563e-06


In [325]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities
for (snt, snt_proba) in result:
    print(f'sequence: {bpe.decode(snt)}, probability: {np.exp(snt_proba)}')

sequence: gun he was and she , probability: 0.19590948446569714
sequence: gun he said not h, probability: 0.09587814834913423
sequence: gun his she was , probability: 0.0659205058685809
sequence: gun had be she wa, probability: 0.054913760604837054
sequence: gun had be she sa, probability: 0.0459949375005684
sequence: gun he saing he , probability: 0.04380554609149823
sequence: gun he said not i, probability: 0.03362248340485744
sequence: gun his she said , probability: 0.03328078729215884
sequence: gun he saing his, probability: 0.025163471271724882
sequence: gun his he was and , probability: 0.022268451271812456


In [327]:
from math import exp
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """

    sigma = 0
    
    snt = [start_token] + snt + [end_token]

    for char in range(len(snt) - 2):
        sigma += log((1 / lm.infer(snt[char], snt[char + 1])[snt[char + 2]]))
    factor = (-1 / float(len(snt))) * sigma
    result = 2 ** factor
    return result 

perplexity(tokenized_text[0], lm)

0.18389452209314572